## get parcel data per apn
provides useful info, especially for development.

does not have any ownership info

In [9]:
import pandas as pd
import time,pathlib,re
import scipy as sp
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [10]:
# get a list of apns
if(1):
    sheetname = "data"
    dfData = pd.read_excel("C:\\Users\\raysa\\docs\\real estate\\sd-tax-auction\\2024-sd-tax-auction\\tax-auction-data.xls",sheet_name=sheetname)
    dfData = df[(df["Auction Type"]!="Timeshare Property") & (df["Canceled"].isnull())]
    apns = df["APN"]
else:
    apns= ["123563454"]

dfData.set_index("APN",inplace=True)

N= np.size(apns)
N

111

https://sites.google.com/chromium.org/driver/downloads?authuser=0

In [11]:
resultFolder = pathlib.Path("C:/work/scratch/temp_sangis")
#resultFolder.mkdir(parents=True, exist_ok=True)


tmpFileName= "{APN}.sangis.html"


In [4]:
help(driver.get)

NameError: name 'driver' is not defined

In [18]:
idx = 0
for apn in apns:
    print("%d (%2.2f) -->%d"%(idx,100.0 *(idx+1)/N,apn));
    idx+=1
    # download the executable
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    #driver  = webdriver.Chrome("C:\work\chrome_driver_110\\chromedriver.exe")
    driver.implicitly_wait(30) # seconds
    url =  'https://gissd.sandag.org/plt/ParcelRpt.aspx?APN={APN}'.format(APN=apn)
    driver.get(url)
    time.sleep(30)
    data = driver.page_source
    driver.close()

    with resultFolder.joinpath(tmpFileName.format(APN=apn)).open('w') as f:
        f.write(data);
print("done")   

0 (0.90) -->2533114000
done


In [14]:
apns[41]

2533114000

# parse html pages saved

now get all the data from html pages and create a composite csv file with one row per apn

In [16]:
mapper = {
#'Property Characteristics:',
'Baths:'                      :3,
'Assessor Land:'              :2,
'Improvements:'               :7,
#'Add. Areas:',
#'Tot. Living Area:',
'Assessor Total:'              :9 ,
'Tax Status:'                 :14,  
'Acreage:'                    :15,
'Bedrooms:'                   :18,
'Tax Rate Area:'              :17,      
 #'District Information:',
 #'City Council:',
 #'Board of Supervisors:',
 #'School District:',
 #'School Union:',
 #'Congress:',
 #'Fire Protection:',
 #'Senate:',
 #'Assembly:',
 #'Community College:',
 #'Planning Areas:',
 #'Census Tract:',
'Subregional Area:'           :67,
 #'Major Statistical Area:',
 #'Community Planning Area:',
'Land Use:'                   :slice(74,75),
'Existing (LU):'              :slice(78,79),
 #'Ecology:',
 #'Vegetation Type (County):',
 #'Vegetation Type (Western Region 2012):',
'Flood Zone/Flood Plain:'     :[89,91]
 #'Geology Type:',
 #'Soil Type:'
}

def processHtml(apn):
    with resultFolder.joinpath(tmpFileName.format(APN=apn)).open('r') as f:
        #try:    
            #f=open(file,'r')
        data = f.read()
            #f.close()
    #except:
    #    print("cant find html for :%s"%(file))
    #    return -1
    soup = BeautifulSoup(data)
  # save a pretty version
    if(0):
        f = open("test.html",'w')
        f.write(soup.prettify());
        f.close()
                               
    ELEM_SEPERATOR = '|'
    ID_IDENTIFIER  = ':'
    s = soup.get_text(ELEM_SEPERATOR)
    startI   = int(s.find("Property Characteristics"))
    endI     = int(s.find("Data Disclaimer"))
    content  = s[startI:endI]
    contentS =content.split(ELEM_SEPERATOR)
    contentId=[c.strip() for c in contentS if ID_IDENTIFIER in c]
    return contentS;                               

In [19]:



df["LU code"]         =''
df["LU Description"]  =''
df["PLU Code"]        =''
df["PLU Description"] =''
df["Land type info"]  =''
for key, idx in mapper.items():
    df[key] = ''


for apn in apns:
    content = processHtml(apn)
    if(content==-1):continue
    #str = "%s,%s,"%(apn,sheetname);
    e = {}
    for key, idx in mapper.items():
        if(isinstance(idx,slice)):
            entry =" ".join(content[idx])
        elif(isinstance(idx,list)):
            entry =" ".join([content[i] for i in idx])
        else:
            entry =content[idx]
        df.loc[apn,key]=entry
    
    flat = "".join(content);
    found = re.search("Land Use.*?LU\)\:([0-9]*)(.*?)\(PLU\)([0-9]*)(.*?)Ecology",flat)
    #str +=",".join(found.groups())+","+found.group (0);
    df.loc[apn,"LU code"]         = found.groups(0)[0]
    df.loc[apn,"LU Description"]  = found.groups(0)[1]
    df.loc[apn,"PLU Code"]        = found.groups(0)[2]
    df.loc[apn,"PLU Description"] = found.groups(0)[3]
    df.loc[apn,"Land type info"]  = found.group(0)
    
df.to_excel("data.xlsx")

In [20]:
df.to_excel(resultFolder/"data.xlsx")